In [1]:
import os
import cv2
import numpy as np
import joblib
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.models import load_model

# Sabitler
MODEL_FILENAME = "best_sign_language_model.joblib"
LE_FILENAME = "label_encoder.joblib"
TF_MODEL_FILENAME = "best_sign_language_model.h5"

# --- 1. MODELİ VE KODLAYICIYI YÜKLEME ---

# Label Encoder'ı yükle
try:
    le = joblib.load(LE_FILENAME)
    class_names = le.classes_
    print(f"[INFO] Label Encoder yüklendi. Sınıflar: {class_names}")
except FileNotFoundError:
    print(f"[HATA] {LE_FILENAME} bulunamadı. Lütfen eğitim kodunu çalıştırın.")
    exit()

# Eğitilmiş modeli yükle (TF veya Scikit-learn)
if os.path.exists(TF_MODEL_FILENAME):
    try:
        model = load_model(TF_MODEL_FILENAME)
        model_type = "TensorFlow ANN"
        print(f"[INFO] TensorFlow modeli ({TF_MODEL_FILENAME}) yüklendi.")
    except Exception as e:
        print(f"[HATA] TensorFlow modeli yüklenirken bir hata oluştu: {e}")
        exit()
elif os.path.exists(MODEL_FILENAME):
    try:
        model = joblib.load(MODEL_FILENAME)
        model_type = "Scikit-learn"
        print(f"[INFO] Scikit-learn modeli ({MODEL_FILENAME}) yüklendi.")
    except Exception as e:
        print(f"[HATA] Scikit-learn modeli yüklenirken bir hata oluştu: {e}")
        exit()
else:
    print(f"[HATA] Kaydedilmiş model ({MODEL_FILENAME} veya {TF_MODEL_FILENAME}) bulunamadı.")
    exit()


# --- 2. MEDIAPIPE VE KAMERA KURULUMU ---

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# MediaPipe Hands modelini başlat
# Canlı akış olduğu için static_image_mode=False
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.5
)

# Kamera başlatma
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("[HATA] Kamera açılamadı.")
    exit()

# Tahmin sonuçları
predicted_label = "Bekleniyor..."

# --- 3. CANLI YAKALAMA DÖNGÜSÜ ---

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("[HATA] Görüntü alınamıyor.")
        continue

    # Performans için görüntüyü yatay çevir (selfie modu)
    image = cv2.flip(image, 1)

    # BGR'dan RGB'ye dönüştür ve sadece okunur yap
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_rgb.flags.writeable = False

    # MediaPipe ile işleme
    results = hands.process(image_rgb)

    # Çizim için tekrar yazılabilir yap
    image_rgb.flags.writeable = True
    image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
    
    # Landmark'ları topla
    landmark_list = []

    if results.multi_hand_landmarks:
        # Algılanan her el için (Bizim modelimiz tek el için)
        for hand_landmarks in results.multi_hand_landmarks:
            
            # Landmark'ları çiz
            mp_drawing.draw_landmarks(
                image, 
                hand_landmarks, 
                mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4), # Nokta rengi
                mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2)  # Çizgi rengi
            )

            # Landmark verilerini model için hazırla
            for lm in hand_landmarks.landmark:
                # x, y, z koordinatlarını ekle
                landmark_list.extend([lm.x, lm.y, lm.z])

            # Modelin tahminini yap
            if len(landmark_list) == 63: # 21 landmark * 3 koordinat
                X_pred = np.array([landmark_list])

                # Tahmini al
                if model_type == "TensorFlow ANN":
                    # TF modeli için tahmin ve argmax (en yüksek olasılıklı sınıf)
                    prediction = model.predict(X_pred, verbose=0)
                    predicted_class_index = np.argmax(prediction[0])
                else:
                    # Scikit-learn modeli için tahmin
                    predicted_class_index = model.predict(X_pred)[0]
                
                # Sayısal indeksi harf etiketine dönüştür
                predicted_label = class_names[predicted_class_index]
            else:
                predicted_label = "Veri Hatası"

    # Tahmin sonucunu ekranda göster
    cv2.putText(
        image, 
        f"Tahmin: {predicted_label}", 
        (10, 30), 
        cv2.FONT_HERSHEY_SIMPLEX, 
        1, # font scale
        (0, 255, 0), # yeşil renk
        2, # kalınlık
        cv2.LINE_AA
    )
    
    cv2.putText(
        image, 
        f"Model: {model_type}", 
        (10, image.shape[0] - 10), 
        cv2.FONT_HERSHEY_SIMPLEX, 
        0.5, # font scale
        (255, 255, 255), # beyaz renk
        1, # kalınlık
        cv2.LINE_AA
    )

    # Görüntüyü göster
    cv2.imshow('ASL Isaret Dili Tanima', image)

    # 'q' tuşuna basılırsa döngüden çık
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

# --- 4. TEMİZLEME ---
hands.close()
cap.release()
cv2.destroyAllWindows()
print("[INFO] Canlı algılama sonlandırıldı.")

2025-10-28 21:24:52.200288: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761675892.245744   39747 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761675892.259318   39747 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-28 21:24:52.381755: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[INFO] Label Encoder yüklendi. Sınıflar: ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R'
 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z' 'del' 'space']


I0000 00:00:1761675895.941625   39747 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4497 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5
I0000 00:00:1761675896.911984   39747 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1761675896.916034   39944 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: Mesa Intel(R) UHD Graphics (CML GT2)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1761675896.952509   39933 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1761675896.974848   39936 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


[INFO] TensorFlow modeli (best_sign_language_model.h5) yüklendi.


W0000 00:00:1761675899.253155   39932 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
I0000 00:00:1761675899.339657   39862 service.cc:148] XLA service 0x7842180055d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1761675899.339799   39862 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 2060, Compute Capability 7.5
2025-10-28 21:24:59.348445: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1761675899.377381   39862 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1761675899.543636   39862 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


[INFO] Canlı algılama sonlandırıldı.
